In [3]:
using GMT

Base.IOError("could not spawn `gmt --version`: no such file or directory (ENOENT)", -2)


┌ Info: Precompiling GMT [5752ebe1-31b9-557e-87aa-f909b540aa54]
└ @ Base loading.jl:1278
   Building Conda → `~/.julia/packages/Conda/sNGum/deps/build.log`
   Building GMT ──→ `~/.julia/packages/GMT/86yOO/deps/build.log`


In [ ]:
# topo = makecpt(color=:plasma, range=(-6000,0,1500), continuous=true);
# grdimage("../bathy/gebco_2020_n45.0_s30.0_w-40.0_e-25.0.nc",
#     shade=(azimuth=250, norm="e0.9"), 
#     J=:M3i, 
#     frame=(axes=:WSen, annot=5, ticks=1,grid=2))

# pscoast!(land="grey")
# psxy!("../bathy/pb2002_boundaries.gmt", lw=0.8)
# basemap!(map_scale="jBR+c37.20+w500+f+o0.4/0.5+l")
# scatter!([-32.3 37.275], fill=:green, marker=:star, markersize=0.2)
# colorbar!(pos=(anchor=:BC, length=(7.5,0.6), horizontal=true, offset=(0,1.0)), 
#     frame=(ylabel=:m,), 
#     show=true)
